## Bevezetés a regresszióba - 1. lecke

#### Helyzetbe helyezés

✅ Számos regressziós módszer létezik, és hogy melyiket választod, az attól függ, milyen kérdésre keresed a választ. Ha például meg szeretnéd jósolni egy adott korú személy valószínű magasságát, akkor `lineáris regressziót` használnál, mivel egy **numerikus értéket** keresel. Ha viszont azt szeretnéd kideríteni, hogy egy adott konyha típusa vegánnak tekinthető-e vagy sem, akkor egy **kategória hozzárendelésére** van szükséged, így `logisztikus regressziót` alkalmaznál. A logisztikus regresszióról később többet fogsz tanulni. Gondolj néhány kérdésre, amelyeket az adatokkal kapcsolatban feltehetsz, és hogy ezek közül melyik módszer lenne a legmegfelelőbb.

Ebben a részben egy [kis adatállománnyal a cukorbetegségről](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) fogsz dolgozni. Képzeld el, hogy egy kezelést szeretnél tesztelni cukorbeteg pácienseken. A gépi tanulási modellek segíthetnek meghatározni, hogy mely páciensek reagálnának jobban a kezelésre, a változók kombinációi alapján. Még egy nagyon alapvető regressziós modell is, ha vizualizáljuk, olyan információkat mutathat a változókról, amelyek segítenek megszervezni az elméleti klinikai vizsgálatokat.

Nos, kezdjünk bele ebbe a feladatba!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Illusztráció: @allison_horst</figcaption>

<!--![Illusztráció: \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Illusztráció: @allison_horst-->


## 1. Eszközkészlet betöltése

Ehhez a feladathoz a következő csomagokra lesz szükségünk:

-   `tidyverse`: A [tidyverse](https://www.tidyverse.org/) egy [R csomagok gyűjteménye](https://www.tidyverse.org/packages), amelyet azért terveztek, hogy a data science gyorsabb, könnyebb és szórakoztatóbb legyen!

-   `tidymodels`: A [tidymodels](https://www.tidymodels.org/) keretrendszer egy [csomaggyűjtemény](https://www.tidymodels.org/packages/) modellezéshez és gépi tanuláshoz.

A következőképpen telepítheted őket:

`install.packages(c("tidyverse", "tidymodels"))`

Az alábbi szkript ellenőrzi, hogy megvannak-e a szükséges csomagok a modul elvégzéséhez, és telepíti őket, ha hiányoznak.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Most töltsük be ezeket a nagyszerű csomagokat, és tegyük elérhetővé őket a jelenlegi R munkamenetünkben. (Ez csak szemléltetés, a `pacman::p_load()` ezt már megtette helyetted)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. A cukorbetegség adatállomány

Ebben a gyakorlatban a regressziós készségeinket fogjuk bemutatni azáltal, hogy előrejelzéseket készítünk egy cukorbetegség adatállomány alapján. A [cukorbetegség adatállomány](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) `442 mintát` tartalmaz a cukorbetegségről, 10 előrejelző változóval: `életkor`, `nem`, `testtömeg-index`, `átlagos vérnyomás`, és `hat vérszérum mérés`, valamint egy kimeneti változóval `y`: a betegség előrehaladásának kvantitatív mértéke egy évvel az alapállapot után.

|Megfigyelések száma|442|
|-------------------|:---|
|Előrejelzők száma|Az első 10 oszlop numerikus előrejelző|
|Kimenet/Cél|A 11. oszlop a betegség előrehaladásának kvantitatív mértéke egy évvel az alapállapot után|
|Előrejelző információ|- életkor években
||- nem
||- bmi testtömeg-index
||- bp átlagos vérnyomás
||- s1 tc, összes szérum koleszterin
||- s2 ldl, alacsony sűrűségű lipoproteinek
||- s3 hdl, magas sűrűségű lipoproteinek
||- s4 tch, összes koleszterin / HDL
||- s5 ltg, valószínűleg a szérum trigliceridszint logaritmusa
||- s6 glu, vércukorszint|

> 🎓 Ne feledd, ez felügyelt tanulás, és szükségünk van egy megnevezett 'y' célváltozóra.

Mielőtt adatokat tudnál manipulálni R-ben, először be kell importálnod az adatokat R memóriájába, vagy létre kell hoznod egy kapcsolatot az adatokkal, amelyet R távolról elérhet.

> A [readr](https://readr.tidyverse.org/) csomag, amely a Tidyverse része, gyors és barátságos módot kínál az adatok téglalap alakú formában történő beolvasására R-be.

Most töltsük be a cukorbetegség adatállományt az alábbi forrás URL-ről: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Ezenkívül végezzünk egy gyors ellenőrzést az adatainkon a `glimpse()` segítségével, és jelenítsük meg az első 5 sort a `slice()` használatával.

Mielőtt továbbmennénk, mutassunk be valamit, amivel gyakran találkozni fogsz R kódban 🥁🥁: a pipe operátort `%>%`

A pipe operátor (`%>%`) logikai sorrendben hajt végre műveleteket azáltal, hogy egy objektumot továbbít egy függvénybe vagy hívási kifejezésbe. Gondolhatsz a pipe operátorra úgy, mintha a kódodban azt mondanád: "és aztán".


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

A `glimpse()` megmutatja, hogy ez az adat 442 sort és 11 oszlopot tartalmaz, az összes oszlop adattípusa pedig `double`.

<br>

> A `glimpse()` és a `slice()` a [`dplyr`](https://dplyr.tidyverse.org/) függvényei. A dplyr, amely a Tidyverse része, egy adatmanipulációs nyelvtan, amely egy következetes igekészletet biztosít, hogy segítsen megoldani a leggyakoribb adatmanipulációs kihívásokat.

<br>

Most, hogy megvan az adat, szűkítsük le egyetlen jellemzőre (`bmi`), amelyet célként használunk ebben a gyakorlatban. Ehhez ki kell választanunk a kívánt oszlopokat. Hogyan tehetjük ezt meg?

A [`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) lehetővé teszi, hogy *kiválasszuk* (és opcionálisan átnevezzük) az oszlopokat egy adatkeretben.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Tanító- és tesztadatok

A felügyelt tanulásban bevett gyakorlat, hogy az adatokat *két részre osztjuk*: egy (általában nagyobb) adathalmazra, amelyet a modell tanítására használunk, és egy kisebb, "visszatartott" adathalmazra, amellyel ellenőrizzük, hogyan teljesített a modell.

Most, hogy az adataink készen állnak, megnézhetjük, hogy egy gép segíthet-e logikus módon szétválasztani a számokat ebben az adathalmazban. Ehhez használhatjuk az [rsample](https://tidymodels.github.io/rsample/) csomagot, amely a Tidymodels keretrendszer része. Ez a csomag lehetővé teszi, hogy létrehozzunk egy objektumot, amely tartalmazza az adatfelosztás *módját*, majd két további rsample függvényt használhatunk a létrehozott tanító- és tesztadatok kinyerésére:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Lineáris regressziós modell tanítása Tidymodels segítségével

Most készen állunk arra, hogy betanítsuk a modellünket!

A Tidymodels-ben a modelleket a `parsnip()` segítségével határozzuk meg, három alapfogalom megadásával:

-   A modell **típusa** különbözteti meg a modelleket, például lineáris regresszió, logisztikus regresszió, döntési fa modellek és így tovább.

-   A modell **módja** olyan általános lehetőségeket tartalmaz, mint a regresszió és a klasszifikáció; egyes modell típusok mindkettőt támogatják, míg mások csak egyetlen módot.

-   A modell **motorja** az a számítási eszköz, amelyet a modell illesztésére használunk. Gyakran ezek R csomagok, mint például **`"lm"`** vagy **`"ranger"`**.

Ez a modellezési információ egy modell specifikációban kerül rögzítésre, úgyhogy építsünk egyet!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Miután egy modellt *meghatároztak*, a modellt `becsülni` vagy `tanítani` lehet a [`fit()`](https://parsnip.tidymodels.org/reference/fit.html) függvény segítségével, általában egy formula és némi adat felhasználásával.

`y ~ .` azt jelenti, hogy `y`-t fogjuk illeszteni mint előrejelzett mennyiséget/célt, amelyet az összes prediktor/jellemző magyaráz, azaz `.` (ebben az esetben csak egy prediktorunk van: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

A modell tanítása során kapott kimenetből láthatjuk a tanulás során meghatározott együtthatókat. Ezek az együtthatók a legjobban illeszkedő egyenes együtthatói, amely a legkisebb összesített hibát adja az aktuális és a becsült változó között.
<br>

## 5. Készítsünk előrejelzéseket a tesztkészleten

Most, hogy betanítottunk egy modellt, használhatjuk azt a betegség előrehaladásának y értékének előrejelzésére a tesztadatkészletben a [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) segítségével. Ez arra szolgál, hogy meghúzzuk a vonalat az adatcsoportok között.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Hurrá! 💃🕺 Épp most tanítottunk be egy modellt, és használtuk előrejelzések készítésére!

Az előrejelzések készítésekor a tidymodels konvenciója szerint mindig egy tibble/adatkeret készül az eredményekkel, szabványosított oszlopnevekkel. Ez megkönnyíti az eredeti adatok és az előrejelzések kombinálását egy használható formátumba, amely alkalmas további műveletekre, például ábrázolásra.

A `dplyr::bind_cols()` hatékonyan összekapcsol több adatkeretet oszlopok mentén.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Modell eredmények ábrázolása

Most itt az ideje, hogy ezt vizuálisan is megnézzük 📈. Készítünk egy szórásdiagramot a tesztkészlet összes `y` és `bmi` értékéről, majd a predikciók segítségével egy vonalat rajzolunk a legmegfelelőbb helyre, a modell adatcsoportosításai közé.

Az R több rendszert kínál grafikonok készítésére, de a `ggplot2` az egyik legelegánsabb és legsokoldalúbb. Ez lehetővé teszi, hogy grafikonokat **független komponensek kombinálásával** állítsunk össze.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Gondolkodj el egy kicsit azon, mi történik itt. Egy egyenes vonal halad át sok apró adatponton, de pontosan mit csinál? Látod, hogyan tudnád ezt a vonalat felhasználni arra, hogy megjósolj egy új, még nem látott adatpont helyét a grafikon y tengelyéhez viszonyítva? Próbáld meg szavakba önteni ennek a modellnek a gyakorlati hasznát.

Gratulálok, elkészítetted az első lineáris regressziós modelledet, létrehoztál vele egy előrejelzést, és megjelenítetted egy grafikonon!



---

**Felelősségkizárás**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális, emberi fordítást igénybe venni. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
